<a href="https://colab.research.google.com/github/1000Roses/NLP_course/blob/main/final_bai1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install lib

In [1]:
!pip install underthesea
!pip install transformers

# Import lib

In [2]:
from underthesea import word_tokenize
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# For Bert
import torch
from transformers import AutoModel, AutoTokenizer

# For read file
import glob
print(glob.glob("/content/drive/MyDrive/NLP/chatbotdata/*.txt"))

['/content/drive/MyDrive/NLP/chatbotdata/giải trí.txt', '/content/drive/MyDrive/NLP/chatbotdata/du lịch.txt', '/content/drive/MyDrive/NLP/chatbotdata/shoping.txt', '/content/drive/MyDrive/NLP/chatbotdata/gia đình.txt', '/content/drive/MyDrive/NLP/chatbotdata/tán gẫu.txt', '/content/drive/MyDrive/NLP/chatbotdata/các câu hỏi phức tạp.txt', '/content/drive/MyDrive/NLP/chatbotdata/đất nước.txt', '/content/drive/MyDrive/NLP/chatbotdata/nghỉ lễ.txt', '/content/drive/MyDrive/NLP/chatbotdata/học tập.txt', '/content/drive/MyDrive/NLP/chatbotdata/thông tin cá nhân.txt', '/content/drive/MyDrive/NLP/chatbotdata/sở thích.txt', '/content/drive/MyDrive/NLP/chatbotdata/trò chuyện về đi ăn.txt', '/content/drive/MyDrive/NLP/chatbotdata/robot.txt', '/content/drive/MyDrive/NLP/chatbotdata/địa chỉ.txt', '/content/drive/MyDrive/NLP/chatbotdata/người yêu.txt', '/content/drive/MyDrive/NLP/chatbotdata/tdtu.txt', '/content/drive/MyDrive/NLP/chatbotdata/bạn bè.txt', '/

# Read file

In [3]:
all_path = glob.glob("/content/drive/MyDrive/NLP/chatbotdata/*.txt")

In [4]:
FRIEND_PRE_DATA = list()
VOCAB_SIZE = 0

In [5]:
for path in all_path:
  with open(path) as frienddata:
    for data in frienddata:
      sent = data.split("__eou__")[:2]
      if sent[0] == ' ' or sent[1] == ' ':
        continue
      if len(sent) == 2:
        anq = list()
        for subsent in sent:
          anq.append(subsent)
        FRIEND_PRE_DATA.append(anq)

In [6]:
df = pd.DataFrame(FRIEND_PRE_DATA, columns=["Question", "Answer"])

In [7]:
df.head(10)

,Question,Answer
0,bạn thích hãng phim nào nhất ?,mình thích hãng phim Marvel nhất
1,thể loại phim bạn thích là gì ?,"mình thích thể loại phim hành động, viễn tưởng"
2,bạn có thần tượng ca sĩ nào không ?,mình rất thích ca sĩ Noo Phước Thịnh
3,bạn có muốn thử vận may vào việc chơi đề ?,mình không muốn cờ bạc
4,tết này bạn có muốn đi xem phim không ?,"tất nhiên rồi, mình rất muốn"
5,cuối tuần này bạn có rãnh hay không?,cuối tuần này mình rãnh
6,bạn có muốn đi xem phim vào cuối tuần này hay ...,dĩ nhiên là muốn rồi
7,mai bạn có muốn đi đá banh chung với tụi mình ...,xin lỗi mai mình có việc bận rồi
8,bạn có dự định đi chơi đâu vào cuối tuần không?,không hiện tại mình chưa có dự định gì
9,bạn cho mình hỏi bạn thích xem phim thể loại gì?,tớ thích xem phim thể loại hành động


In [8]:
document_token = [ " ".join([ e.replace(' ','_') for e in word_tokenize(sent.lower())]) for sent in df['Question']]
document_token_list = [word_tokenize(sent.lower()) for sent in df['Question']]  
question_lower = [question.lower() for question in df['Question']]

In [9]:
df["Word_segmentation"] = document_token
df["Tokenize_list"] = document_token_list
df['Question'] = question_lower

In [10]:
df.head(10)

,Question,Answer,Word_segmentation,Tokenize_list
0,bạn thích hãng phim nào nhất ?,mình thích hãng phim Marvel nhất,bạn thích hãng phim nào nhất ?,"[bạn, thích, hãng, phim, nào, nhất, ?]"
1,thể loại phim bạn thích là gì ?,"mình thích thể loại phim hành động, viễn tưởng",thể_loại phim bạn thích là gì ?,"[thể loại, phim, bạn, thích, là, gì, ?]"
2,bạn có thần tượng ca sĩ nào không ?,mình rất thích ca sĩ Noo Phước Thịnh,bạn có thần_tượng ca_sĩ nào không ?,"[bạn, có, thần tượng, ca sĩ, nào, không, ?]"
3,bạn có muốn thử vận may vào việc chơi đề ?,mình không muốn cờ bạc,bạn có muốn thử vận may vào việc chơi đề ?,"[bạn, có, muốn, thử, vận, may, vào, việc, chơi..."
4,tết này bạn có muốn đi xem phim không ?,"tất nhiên rồi, mình rất muốn",tết này bạn có muốn đi xem phim không ?,"[tết, này, bạn, có, muốn, đi, xem, phim, không..."
5,cuối tuần này bạn có rãnh hay không?,cuối tuần này mình rãnh,cuối tuần này bạn có rãnh hay không ?,"[cuối, tuần, này, bạn, có, rãnh, hay, không, ?]"
6,bạn có muốn đi xem phim vào cuối tuần này hay ...,dĩ nhiên là muốn rồi,bạn có muốn đi xem phim vào cuối tuần này hay ...,"[bạn, có, muốn, đi, xem, phim, vào, cuối, tuần..."
7,mai bạn có muốn đi đá banh chung với tụi mình ...,xin lỗi mai mình có việc bận rồi,mai bạn có muốn đi đá banh chung với tụi_mình ...,"[mai, bạn, có, muốn, đi, đá, banh, chung, với,..."
8,bạn có dự định đi chơi đâu vào cuối tuần không?,không hiện tại mình chưa có dự định gì,bạn có dự_định đi chơi đâu vào cuối tuần không ?,"[bạn, có, dự định, đi, chơi, đâu, vào, cuối, t..."
9,bạn cho mình hỏi bạn thích xem phim thể loại gì?,tớ thích xem phim thể loại hành động,bạn cho mình hỏi bạn thích xem phim thể_loại gì ?,"[bạn, cho, mình, hỏi, bạn, thích, xem, phim, t..."


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
countVectorizer_sent = CountVectorizer()
countVectorizer_corpus = CountVectorizer()

In [13]:
model_countVectorizer_sent = countVectorizer_sent.fit(df['Word_segmentation'])
model_countVectorizer_corpus = countVectorizer_sent.fit_transform(df['Word_segmentation'])

In [14]:
tfidfVectorizer_sent = TfidfVectorizer()
tfidfVectorizer_corpus = TfidfVectorizer()

In [15]:
model_tfidfVectorizer_sent  = tfidfVectorizer_sent.fit(df['Word_segmentation'])
model_tfidfVectorizer_corpus = tfidfVectorizer_corpus.fit_transform(df['Word_segmentation'])

In [16]:
binaryVectorizer_sent = CountVectorizer(binary=True)
binaryVectorizer_corpus = CountVectorizer(binary=True)

In [17]:
model_binaryVectorizer_sent = binaryVectorizer_sent.fit(df['Word_segmentation'])
model_binaryVectorizer_corpus = binaryVectorizer_corpus.fit_transform(df['Word_segmentation'])

In [18]:
def ir_countVectorizer(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_vector = model_countVectorizer_sent.transform([question]).toarray()[0]

  r = list()
  for vector in model_countVectorizer_corpus.toarray():
    r.append(cosine_similarity(np.array([q_vector]), np.array([vector]))[0][0])
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]


In [19]:
def ir_tfidfVectorizer(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_vector = model_tfidfVectorizer_sent.transform([question]).toarray()[0]

  r = list()
  for vector in model_tfidfVectorizer_corpus.toarray():
    r.append(cosine_similarity(np.array([q_vector]), np.array([vector]))[0][0])
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]

In [20]:
def ir_tfidfVectorizer(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_vector = model_binaryVectorizer_sent.transform([question]).toarray()[0]

  r = list()
  for vector in model_binaryVectorizer_corpus.toarray():
    r.append(cosine_similarity(np.array([q_vector]), np.array([vector]))[0][0])
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]

In [21]:
def ir_binary(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_vector = model_tfidfVectorizer_sent.transform([question]).toarray()[0]

  r = list()
  for vector in model_tfidfVectorizer_corpus.toarray():
    r.append(cosine_similarity(np.array([q_vector]), np.array([vector]))[0][0])
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]

In [22]:
w2v_model = Word2Vec(df['Tokenize_list'], size = 300, min_count = 2, workers = 3)

In [23]:
embedding_each_sent = list()
for sent_tokenize in df['Tokenize_list']:
  embedding = list()
  for word in sent_tokenize:
    if word not in w2v_model.wv.vocab:
      #300 dimension is the same as size of output embedding word2vec
      embedding.append(np.random.rand(300))  
    else:
      embedding.append(w2v_model.wv.word_vec(word))
  embedding = np.mean(embedding, axis = 0)
  embedding_each_sent.append(embedding)

In [24]:
df['Gensim_embedding'] = embedding_each_sent

In [25]:
def ir_gensim(question):
  question = question.lower()
  tokenize_q = word_tokenize(question)
  q_embedding = list()
  for word in tokenize_q:
    if word not in w2v_model.wv.vocab:
        #300 dimension is the same as size of output embedding word2vec
      q_embedding.append(np.random.rand(300))  
    else:
      q_embedding.append(w2v_model.wv.word_vec(word))
  q_embedding = np.mean(q_embedding, axis = 0)

  r = list()
  for embedding in df['Gensim_embedding']:
    r.append(cosine_similarity(np.array([q_embedding]), np.array([embedding]))[0][0])
    
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]



# BERT

In [26]:
import torch
from transformers import AutoModel, AutoTokenizer

In [27]:
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")

# For transformers v4.x+: 
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [28]:
cos_bert = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

In [29]:
phobert_id_each_sent = [ phobert_tokenizer.encode(line) for line in df['Word_segmentation']]

In [30]:
# convert list to torch tensor format
phobert_id_each_sent =  [torch.tensor([sent]) for sent in phobert_id_each_sent]

In [31]:
def bert_embedding(phobert_id_each_sent):
  r = list()
  i = 0
  for sent in phobert_id_each_sent:
    # with torch.no_grad():
    embedding = phobert_model(sent)[0][0]
    r.append(np.mean(embedding.detach().numpy() , axis = 0))
  return r


In [32]:
phobert_model(phobert_id_each_sent[0])[0][0].detach().numpy()

array([[ 2.3426484e-01,  2.9970169e-01,  3.3295877e-02, ...,
        -3.6920020e-01,  2.6532916e-02,  2.3787358e-01],
       [-7.9690451e-03, -3.7074766e-01,  8.3288543e-02, ...,
        -1.5157704e-01,  6.5741909e-04,  6.8149328e-01],
       [ 9.0758212e-02,  4.4950882e-01, -1.5434603e-01, ...,
        -2.6773065e-01, -2.4946736e-01,  5.5675024e-01],
       ...,
       [ 3.6425075e-01,  2.7860841e-01, -1.6791721e-01, ...,
        -2.2941221e-01,  2.4586850e-01,  3.3459982e-01],
       [ 1.2696093e-01,  3.8982850e-01, -1.9373862e-01, ...,
        -5.1740015e-01,  1.2289950e-01,  6.0243094e-01],
       [ 1.5069169e-01,  2.4493612e-01,  1.5941775e-01, ...,
        -3.2291153e-01, -1.9354339e-01,  1.8636650e-01]], dtype=float32)

In [33]:
embedding_bert = bert_embedding(phobert_id_each_sent)

In [34]:
np.array(embedding_bert).shape

(5852, 768)

In [35]:
df['Bert_embedding'] = embedding_bert

In [36]:
df.head(10)

,Question,Answer,Word_segmentation,Tokenize_list,Gensim_embedding,Bert_embedding
0,bạn thích hãng phim nào nhất ?,mình thích hãng phim Marvel nhất,bạn thích hãng phim nào nhất ?,"[bạn, thích, hãng, phim, nào, nhất, ?]","[-0.0881896, -0.016349202, -0.35377175, -0.160...","[0.09159158, 0.24511947, -0.0973822, -0.332693..."
1,thể loại phim bạn thích là gì ?,"mình thích thể loại phim hành động, viễn tưởng",thể_loại phim bạn thích là gì ?,"[thể loại, phim, bạn, thích, là, gì, ?]","[-0.08587737, -0.016290462, -0.3477278, -0.155...","[-0.01886774, 0.19260995, -0.04660156, -0.5031..."
2,bạn có thần tượng ca sĩ nào không ?,mình rất thích ca sĩ Noo Phước Thịnh,bạn có thần_tượng ca_sĩ nào không ?,"[bạn, có, thần tượng, ca sĩ, nào, không, ?]","[-0.07320876, -0.0122848395, -0.2983003, -0.13...","[0.12758176, -0.017866714, -0.14962307, -0.553..."
3,bạn có muốn thử vận may vào việc chơi đề ?,mình không muốn cờ bạc,bạn có muốn thử vận may vào việc chơi đề ?,"[bạn, có, muốn, thử, vận, may, vào, việc, chơi...","[-0.022800383024037398, 0.09819823832119891, -...","[-0.1491277, 0.4655893, -0.0659392, -0.2527817..."
4,tết này bạn có muốn đi xem phim không ?,"tất nhiên rồi, mình rất muốn",tết này bạn có muốn đi xem phim không ?,"[tết, này, bạn, có, muốn, đi, xem, phim, không...","[-0.09164353, -0.014625112, -0.38200697, -0.17...","[-0.009907469, 0.09272019, 0.009779289, -0.287..."
5,cuối tuần này bạn có rãnh hay không?,cuối tuần này mình rãnh,cuối tuần này bạn có rãnh hay không ?,"[cuối, tuần, này, bạn, có, rãnh, hay, không, ?]","[-0.08381217, -0.012829467, -0.3441187, -0.159...","[0.12912, 0.3531937, 0.090220764, -0.18896861,..."
6,bạn có muốn đi xem phim vào cuối tuần này hay ...,dĩ nhiên là muốn rồi,bạn có muốn đi xem phim vào cuối tuần này hay ...,"[bạn, có, muốn, đi, xem, phim, vào, cuối, tuần...","[-0.08794381, -0.014473697, -0.3639611, -0.170...","[-0.0017565131, 0.19291644, -0.092288755, -0.1..."
7,mai bạn có muốn đi đá banh chung với tụi mình ...,xin lỗi mai mình có việc bận rồi,mai bạn có muốn đi đá banh chung với tụi_mình ...,"[mai, bạn, có, muốn, đi, đá, banh, chung, với,...","[-0.072087236, -0.011021971, -0.2984917, -0.13...","[-0.10028739, 0.28775898, -0.061872803, -0.266..."
8,bạn có dự định đi chơi đâu vào cuối tuần không?,không hiện tại mình chưa có dự định gì,bạn có dự_định đi chơi đâu vào cuối tuần không ?,"[bạn, có, dự định, đi, chơi, đâu, vào, cuối, t...","[-0.08064279, -0.012735165, -0.3323268, -0.154...","[-0.017464964, 0.10116356, 0.06738061, -0.2574..."
9,bạn cho mình hỏi bạn thích xem phim thể loại gì?,tớ thích xem phim thể loại hành động,bạn cho mình hỏi bạn thích xem phim thể_loại gì ?,"[bạn, cho, mình, hỏi, bạn, thích, xem, phim, t...","[-0.08316428, -0.016183775, -0.3473207, -0.153...","[-0.08897535, 0.1993293, 0.13860983, -0.296447..."


In [37]:
def ir_bert(question):
  # Tokenize
  question = " ".join([ token.replace(" ", "_")  for token in word_tokenize(question.lower())])
  q_id = torch.tensor([phobert_tokenizer.encode(question)])
  
  q_embedding = phobert_model(q_id)[0][0]
  q_embedding = np.mean(q_embedding.detach().numpy() , axis = 0)

  r = list()
  for vector in df['Bert_embedding']:
    cos = cos_bert(torch.Tensor([q_embedding]), torch.Tensor([vector]))
    cos = cos.detach().numpy()[0]
    r.append(cos)
  
  r = np.array(r)
  i = np.argmax(r)
  return df['Answer'][i]

In [38]:
def ir():
  while(True):
    q = input("Nhập question: ")
    if q == "q":
      break

    print("bot with count_vectorize: ", ir_countVectorizer(q))
    print("bot with tfidf_vectorize: ", ir_tfidfVectorizer(q))
    print("bot with binary_vectorize: ", ir_binary(q))
    print("bot with gensim_embedding: ", ir_gensim(q))
    print('bot with bert_embedding: ', ir_bert(q))
    

In [39]:
ir()

Nhập question: chào bạn nha
bot with count_vectorize:   Chào bạn, mình tên là Hùng 
bot with tfidf_vectorize:   Tôi ổn. còn bạn thì Sao? 
bot with binary_vectorize:   Chào bạn, mình tên là Hùng 
bot with gensim_embedding:   có 
bot with bert_embedding:   Không có gì đâu 
Nhập question: dạo này bạn có khỏe không thế?
bot with count_vectorize:   mình có đi đà lạt hồi noel 2020 
bot with tfidf_vectorize:   mình có đi đà lạt hồi noel 2020 
bot with binary_vectorize:   Đang chán đời cậu ơi, muốn giải tỏa. 
bot with gensim_embedding:   không, mình chỉ có màu xanh thôi 
bot with bert_embedding:   Đang chán đời cậu ơi, muốn giải tỏa. 
Nhập question: q
